In [1]:
import camelot
import pandas as pd
import re
import numpy as np

In [3]:
regex1 = re.compile(r'^Z\d|^PZ\d')

regex2 = re.compile(r'[A-Z]{3}')

regex3 = re.compile(r'^NY|^jfr|^[XYZUVW]\d')

regex4 = re.compile(r'^\d|^<')

delar = []

for i in range(302, 304):
    # Läs in sidan i från LNU:s kodbok
    tables = camelot.read_pdf('LNU{}.pdf'.format(i))
    
    print(i)
    # Gör om "tabellen" på sidan till en DataFrame. Vissa sidor, t.ex. 36, har ingen tabell alls. Kolla därför att tabell finns.
    if tables.n != 0:
        df = tables[0].df

        # Dela upp strängen i varje cell vid "\n".
        L1 = [df.loc[i][0].split('\n') for i in df.index]
    
        # Skapa lista med namn (se exempel längre upp för förklaring av regex)
        namn = []

        for j in range(len(L1)):
            for k in range(len(L1[j])):
                a = len(namn)
                if bool(regex1.findall(L1[j][k])) == True:
                    namn.append(L1[j][k])
                    break
            if len(namn) == a:
                namn.append('sidbrytning')
        
        
        # Skapa listan med beskrivningar.         
        beskrivning = []

        for j in range(len(L1)):
            for k in range(len(L1[j])):
                a = len(beskrivning)
                if bool(regex2.findall(L1[j][k])) == True:
                    beskrivning.append(L1[j][k])
                    break
            if len(beskrivning) == a:
                beskrivning.append('sidbrytning')
        
        # Skapa listan med enkättexter.
        enkättext = []

        for j in range(len(L1)):
            textdelar = []
            for k in range(3, len(L1[j])):
                if bool(regex4.findall(L1[j][k])) == True:
                    break
                if bool(regex3.findall(L1[j][k])) == False:
                    textdelar.append(L1[j][k])
            enkättext.append(''.join(textdelar))
        
        # Skapa en dataframe av listorna namn, beskrivning och enkättext. 
        df = pd.DataFrame.from_dict({'Variabelnamn': namn, 'Beskrivning': beskrivning, 'Enkättext': enkättext})
    
        # Nu har hela sidan i blivit en dataframe och läggs i en lista som heter "delar".
        delar.append(df)

302
303


In [4]:
LNU = pd.concat(delar)

In [5]:
LNU

,Variabelnamn,Beskrivning,Enkättext
0,sidbrytning,Vårdförbundet (SHSTF),
1,Z831,FACKLIG MÖTESAKTIVITET,Har du varit på något fackföreningsmöte under ...
2,Z832,FACKLIGA UPPDRAG,Har du eller har du haft någon förtroende...


In [7]:
df

,Variabelnamn,Beskrivning,Enkättext
0,sidbrytning,Vårdförbundet (SHSTF),
1,Z831,FACKLIG MÖTESAKTIVITET,Har du varit på något fackföreningsmöte under ...
2,Z832,FACKLIGA UPPDRAG,Har du eller har du haft någon förtroende...


In [9]:
tables = camelot.read_pdf('LNU302.pdf')

<TableList n=0>

In [10]:
tables

<TableList n=0>

In [16]:
df = pd.DataFrame([[1, 2, 3], [4, 5, 6]], columns=list('ABC'))
df

,A,B,C
0,1,2,3
1,4,5,6


In [27]:
row = pd.Series({'A': 'Z830', 'B': 'FACKFÖRBUND', 'C': 'Vilket förbund (eller liknande) tillhör du?'}, name = 'hej')
df.append(row)

,A,B,C
0,1,2,3
1,4,5,6
hej,Z830,FACKFÖRBUND,Vilket förbund (eller liknande) tillhör du?
